In [42]:
from hangman import hangman, get_dictionary
import wordfreq
import json
from decimal import Decimal, getcontext

In [48]:

# Makes the dictionary with the word frequencies
big_dictionary = get_dictionary("words.txt")

# Convert to a dictionary with higher precision
freq_dict = {}
for word in big_dictionary:
    word = word.lower()
    weight = Decimal(wordfreq.word_frequency(word, "en", wordlist='small', minimum=0.0))
    if weight > 0.0 and len(word) > 1:
        freq_dict[word] = float(weight)  # Store as float to ensure JSON compatibility

# Write it to a JSON file with high precision
with open('webppl-model/word_freq.json', 'w') as f:
    json.dump(freq_dict, f, indent=2)


In [49]:
getcontext().prec = 50
weight = Decimal(wordfreq.word_frequency("moonrise", "en", wordlist='small', minimum=0.0))
print(weight)
print(weight > 0.0)

0
False


In [50]:
#get the letter frequencies from freq_dict 

letter_freq = {}
for word in freq_dict:
    for letter in word:
        if letter in letter_freq:
            letter_freq[letter] += freq_dict[word]
        else:
            letter_freq[letter] = freq_dict[word]
            
#normalize it 
total = sum(letter_freq.values())
for letter in letter_freq:
    letter_freq[letter] /= total
    
# Write it to a JSON file with high precision
with open('webppl/letter_freq.json', 'w') as f:
    json.dump(letter_freq, f, indent=2)

FileNotFoundError: [Errno 2] No such file or directory: 'webppl/letter_freq.json'

In [46]:
""" turn csv into data of the form 

var partial = ["_", "o", "_", "_", "e", "_", "_", "_", "_"]
var in_letters = ["o", "e"]
var not_in_letters = ["s"]
var previousGuesses = in_letters.concat(not_in_letters)

var data = [
  {
    previousGuesses: previousGuesses,
    actualGuess: "l",
    partialWordPattern: partial,
    word: "wonderful"
  },
]


where each guess is piece of data. each row of the csv is a series of guesses eg. 
8	calendar	e	t 	a	l	n	i	o	s	r	c	d	
""" 
import csv
import json

def make_data_from_csv(csv_file_path, data_file_path):
    # Helper function to update the partial word pattern
    def update_partial(partial, word, guess):
        return [char if char == guess or char in partial else "_" for char in word]

    # Initialize variables
    output_data = []

    # Read the CSV file
    with open(csv_file_path, "r") as csv_file:
        csv_reader = csv.reader(csv_file, delimiter="\t")
        next(csv_reader)  # Skip the first row (header)

        for row in csv_reader:
            row = row[0].split(",")
            
            if len(row) < 4 or row[0] == "":
                continue

            # Parse the data from the row
            step_number, word, *guesses = row
            word = word.strip().lower()
            broken_words = ["moonrise"]
            if word in broken_words:
                continue
            guesses = [guess.strip() for guess in guesses if guess.strip()]

            # Initialize variables for this row
            partial = ["_" for _ in word]
            in_letters = []
            not_in_letters = []

            # Process each guess in the row
            for i, guess in enumerate(guesses):
                # Create the data point BEFORE applying the current guess
                data_point = {
                    "previousGuesses": in_letters + not_in_letters,
                    "actualGuess": guess,
                    "partialWordPattern": partial.copy(),  # Partial before applying the guess
                    "word": word
                }

                # Append the data point
                output_data.append(data_point)

                # Update the partial and letter lists AFTER creating the data point
                if guess in word:
                    in_letters.append(guess)
                    partial = update_partial(partial, word, guess)
                else:
                    not_in_letters.append(guess)

    # Write the output data to a JSON file
    with open(data_file_path, "w") as output_file:
        json.dump(output_data, output_file, indent=2)



In [53]:
make_data_from_csv("dataset.csv", "webppl-model/data.js")

In [51]:
# Separate successful and unsuccessful guess data
def make_successful_unsuccessful_data(data_file_path, successful_data_file_path, unsuccessful_data_file_path):
    with open(data_file_path, "r") as data_file:
        data = json.load(data_file)

    # Filter successful and unsuccessful guesses
    successful_data = [
        {
            "previousGuesses": d["previousGuesses"],
            "actualGuess": d["actualGuess"],
            "partialWordPattern": [char if char in d["previousGuesses"] else "_" for char in d["word"]],
            "word": d["word"]
        }
        for d in data if d["actualGuess"] in d["word"]
    ]

    unsuccessful_data = [
        {
            "previousGuesses": d["previousGuesses"],
            "actualGuess": d["actualGuess"],
            "partialWordPattern": [char if char in d["previousGuesses"] else "_" for char in d["word"]],
            "word": d["word"]
        }
        for d in data if d["actualGuess"] not in d["word"]
    ]

    # Save successful guesses
    with open(successful_data_file_path, "w") as output_file:
        json.dump(successful_data, output_file, indent=2)

    # Save unsuccessful guesses
    with open(unsuccessful_data_file_path, "w") as output_file:
        json.dump(unsuccessful_data, output_file, indent=2)


In [52]:
make_successful_unsuccessful_data("webppl-model/data.js", "webppl-model/successful_data.js", "webppl-model/unsuccessful_data.js")